# Affine Cipher

The affine cipher is actually the multiplicative cipher combined with the Caesar cipher, and the multiplicative cipher is similar to the Caesar cipher except it uses multiplication instead of addition to encrypt messages. Because the affine cipher uses two different ciphers as part of its encryption process, it needs two keys: one for the multiplicative cipher and another for the Caesar cipher.

In [1]:
from cracking_codes.utils import cryptomath
import random

In [2]:
SYMBOLS = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz1234567890 !?.'

In [3]:
def getKeyParts(key):
    keyA = key // len(SYMBOLS)
    keyB = key % len(SYMBOLS)
    return (keyA, keyB)


def checkKeys(keyA, keyB, mode):
    if keyA == 1 and mode == 'encrypt':
        sys.exit('Cipher is weak if key A is 1. Choose a different key.')
    if keyB == 0 and mode == 'encrypt':
        sys.exit('Cipher is weak if key B is 0. Choose a different key.')
    if keyA < 0 or keyB < 0 or keyB > len(SYMBOLS) - 1:
        sys.exit('Key A must be greater than 0 and Key B must be between 0 and %s.' % (len(SYMBOLS) - 1))
    if cryptomath.gcd(keyA, len(SYMBOLS)) != 1:
        sys.exit('Key A (%s) and the symbol set size (%s) are not relatively prime. Choose a different key.' % (keyA, len(SYMBOLS)))


def encryptAffineCipher(key, message):
    keyA, keyB = getKeyParts(key)
    checkKeys(keyA, keyB, 'encrypt')
    ciphertext = ''
    for symbol in message:
        if symbol in SYMBOLS:
            symbolIndex = SYMBOLS.find(symbol)
            ciphertext += SYMBOLS[(symbolIndex * keyA + keyB) % len(SYMBOLS)]
        else:
            ciphertext += symbol
    return ciphertext


def decryptAffineCipher(key, message):
    keyA, keyB = getKeyParts(key)
    checkKeys(keyA, keyB, 'decrypt')
    plaintext = ''
    modInverseOfKeyA = cryptomath.findModInverse(keyA, len(SYMBOLS))

    for symbol in message:
        if symbol in SYMBOLS:
            symbolIndex = SYMBOLS.find(symbol)
            plaintext += SYMBOLS[(symbolIndex - keyB) * modInverseOfKeyA % len(SYMBOLS)]
        else:
            plaintext += symbol
    return plaintext


def getRandomKey():
    while True:
        keyA = random.randint(2, len(SYMBOLS))
        keyB = random.randint(2, len(SYMBOLS))
        if cryptomath.gcd(keyA, len(SYMBOLS)) == 1:
            print(keyA * len(SYMBOLS) + keyB)
            return keyA * len(SYMBOLS) + keyB

In [4]:
encryptAffineCipher(getRandomKey(), "Hello, World")

1917


'IPUUp,Tvp U1'

In [5]:
decryptAffineCipher(1917, 'IPUUp,Tvp U1')

'Hello, World'

In [6]:
from cracking_codes.utils import detect_english

In [7]:
def hackAffineCipher(message, silent=True):
    for key in range(len(SYMBOLS) ** 2):
        keyA = getKeyParts(key)[0]
        if cryptomath.gcd(keyA, len(SYMBOLS)) != 1:
            continue

        decryptedText = decryptAffineCipher(key, message)
        if not silent:
            print(f'Trying key #{key}...')

        if detect_english.isEnglish(decryptedText):
            print()
            print('Possible encryption hack:')
            print('Key:',key)
            print('Decrypted message: ' + decryptedText[:200])
            print()

In [8]:
hackAffineCipher('IPUUp,Tvp U1')


Possible encryption hack:
Key: 1917
Decrypted message: Hello, World


Possible encryption hack:
Key: 3133
Decrypted message: duLL ,SU vLz

